In [1]:
import torch
from torch.nn.functional import pad

In [2]:
dim = 2
# mock embedding matrix 1
seq_len1 = 5
matrix1 = torch.randn(seq_len1, dim)
# mock embedding matrix 2
seq_len2 = 3
matrix2 = torch.randn(seq_len2, dim)
# ensure matrices only contain non-negative values
matrix1 = torch.abs(matrix1)
matrix2 = torch.abs(matrix2)

In [3]:
matrices = [matrix1, matrix2]

In [4]:
longest_seq_len = max([matrix.size(0) for matrix in matrices])

In [5]:
seq_lens = [matrix.size(0) for matrix in matrices]

In [6]:
for i in range(len(matrices)):
    m = matrices[i]
    matrices[i] = pad(m, (0, 0, 0, longest_seq_len - m.size(0)), value=-100000)

In [7]:
matrices[-1][-1] = 1

In [8]:
from torch.nn.functional import scaled_dot_product_attention as sdpa

In [9]:
attn_matrices = []

In [10]:
naive_attn = []

In [11]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    causal_mask[:, seq_lens[i]: ] = 0
    attn_matrices.append(causal_mask.bool())

In [12]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    naive_attn.append(causal_mask.bool())

In [13]:
attn_matrices = torch.stack(attn_matrices)
naive_attn = torch.stack(naive_attn)

In [14]:
matrices = torch.stack(matrices)

In [15]:
q = torch.rand_like(matrices)
k = torch.rand_like(matrices)

In [16]:
result = sdpa(q, k, matrices, naive_attn)

In [17]:
matrices

tensor([[[ 7.3774e-01,  7.5188e-01],
         [ 1.4557e-01,  1.7227e-01],
         [ 2.2988e-01,  4.0861e-01],
         [ 6.7407e-01,  1.0262e+00],
         [ 8.1342e-01,  1.5773e+00]],

        [[ 1.3564e+00,  1.6196e+00],
         [ 1.8877e-01,  3.0078e-01],
         [ 5.4786e-01,  3.8402e-01],
         [-1.0000e+05, -1.0000e+05],
         [ 1.0000e+00,  1.0000e+00]]])

In [18]:
result

tensor([[[ 7.3774e-01,  7.5188e-01],
         [ 4.7628e-01,  4.9596e-01],
         [ 4.1083e-01,  4.7393e-01],
         [ 4.7637e-01,  6.3472e-01],
         [ 5.3254e-01,  7.9940e-01]],

        [[ 1.3564e+00,  1.6196e+00],
         [ 7.2223e-01,  9.0334e-01],
         [ 6.5259e-01,  7.0348e-01],
         [-2.8265e+04, -2.8265e+04],
         [-2.2341e+04, -2.2341e+04]]])

In [19]:
sdpa(q, k, matrices, attn_matrices)

tensor([[[0.7377, 0.7519],
         [0.4763, 0.4960],
         [0.4108, 0.4739],
         [0.4764, 0.6347],
         [0.5325, 0.7994]],

        [[1.3564, 1.6196],
         [0.7222, 0.9033],
         [0.6526, 0.7035],
         [0.6009, 0.6265],
         [0.6329, 0.6768]]])